In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0019953250885009766
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 19.54636721801739
200000 30.631796417644154
300000 34.684065649915745
400000 39.90630894785882
500000 43.19753658291805
600000 55.67468906439994
700000 53.954554034943094
800000 50.74459126151645
900000 47.446599767131964
1000000 35.17933275421004
1100000 32.84187567285649
1200000 50.702995381171725
1300000 55.484808017510545
1400000 58.69596016633865
1500000 60.39203027348639
1600000 61.423191165582416
1700000 54.262393335412284
1800000 48.299138982658675
1900000 58.28378295090873
2000000 37.32198582287965
2100000 29.951212605872463
2200000 64.03704114508007
2300000 53.357364426077
2400000 53.13484381742498
2500000 52.48521216240287
2600000 56.90407721228832
2700000 58.813258963785934
2800000 62.03997207607566
2900000 55.92798476048677
3000000 47.65445

30000000 77.91892768839433
30100000 77.87145404317573
30200000 74.82509845232302
30300000 32.21860730792632
30400000 76.64151802694904
30500000 74.24798277028147
30600000 78.31260107014039
30700000 76.05666605860195
30800000 69.01379974911458
30900000 66.67573512152094
31000000 52.85575261502667
31100000 39.878692603294176
31200000 68.47945573355493
31300000 69.88302123761297
31400000 72.15319305070774
31500000 74.00573497020791
31600000 76.28806305838695
31700000 74.23799051399553
31800000 44.03161073627534
31900000 69.47158496354538
32000000 80.78972093094148
32100000 70.88322190242384
32200000 74.77157479162578
32300000 34.12932780569837
32400000 71.73000937027753
32500000 76.30955326416324
32600000 70.40844466129954
32700000 68.90805727956374
32800000 66.88551790194815
32900000 64.27404398895249
33000000 52.744256055438456
33100000 37.92853106416016
33200000 63.201812309185236
33300000 67.06547855536205
33400000 69.55969397256708
33500000 74.3389642442709
33600000 71.94468846780036

60400000 68.48740401963546
60500000 45.60893027043631
60600000 47.298217010302345
60700000 61.40324152392774
60800000 62.63856524950675
60900000 61.57692074052171
61000000 51.9453023558756
61100000 35.457838618655835
61200000 61.91454521056817
61300000 65.01959709754337
61400000 61.141469455726195
61500000 63.58819752251597
61600000 31.517451555954153
61700000 66.9372089518879
61800000 71.2302991638271
61900000 67.08064260434088
62000000 77.37593299246998
62100000 77.87095720243404
62200000 74.20805043882297
62300000 70.25381983641348
62400000 72.25066004125004
62500000 55.362159740771844
62600000 40.85671339153024
62700000 68.15598767415757
62800000 67.65092764890349
62900000 66.11164866463864
63000000 54.799449036716375
63100000 32.63019163312234
63200000 64.63991475670672
63300000 62.48208394051847
63400000 67.9951014031455
63500000 58.711584444166775
63600000 38.03212691333234
63700000 66.01116070147941
63800000 71.83734616427968
63900000 69.90507815408733
64000000 74.6877842005049

90700000 63.452676813028354
90800000 27.71913476814507
90900000 62.26471442250557
91000000 56.22649063314565
91100000 30.53076433794857
91200000 61.57045976627867
91300000 52.67607291725196
91400000 41.1846005437135
91500000 60.814579075652865
91600000 61.751830461307776
91700000 63.070406881140705
91800000 66.98561453699313
91900000 69.13696575443987
92000000 69.49439895071853
92100000 70.4187053678434
92200000 64.0517198021684
92300000 63.63067804496876
92400000 62.00650470246914
92500000 60.08693075626878
92600000 59.742966047180154
92700000 58.74883456004206
92800000 27.435207058260396
92900000 58.62022019526603
93000000 52.13287025131719
93100000 31.880680438332767
93200000 58.181930116874476
93300000 40.80972291090879
93400000 48.281881701185114
93500000 58.185686617677405
93600000 58.800235112116596
93700000 61.02720869004787
93800000 62.38318252954292
93900000 66.20971843945286
94000000 62.82089230135481
94100000 68.88926372295148
94200000 72.4871728093738
94300000 67.653426136

120100000 82.16499945440441
120200000 77.5018908826716
120300000 75.5462194722808
120400000 70.2784579855558
120500000 74.7341273392699
120600000 71.38315567341166
120700000 72.26698313792805
120800000 70.68733458640779
120900000 73.10888411868032
121000000 37.36197241062594
121100000 10.435486414385355
121200000 61.29262450623179
121300000 68.28258686945549
121400000 68.47451997990586
121500000 73.84934389291682
121600000 71.32244441267203
121700000 73.791187799386
121800000 72.26398316447025
121900000 75.42839993667327
122000000 81.2884730475123
122100000 24.073356281569335
122200000 7.120980301274668
122300000 7.347119223041962
122400000 7.3105138104693514
122500000 6.53780042934496
122600000 7.164001211276452
122700000 6.440378151672412
122800000 6.745802834096134
122900000 6.368874866267055
123000000 6.458630229652523
123100000 6.916991340401518
123200000 6.418530227820722
123300000 6.306313405093848
123400000 6.827028898805743
123500000 7.228979594336529
123600000 8.4232183203571

...

884700000 2.623981953482295
884800000 3.5430278993468556
884900000 2.151711888427669
885000000 3.1337762013317754
885100000 2.35315880924385
885200000 3.059172271539277
885300000 3.0211792581952346
885400000 2.3137510152316025
885500000 3.2506131531678557
885600000 2.075952146851183
885700000 3.412996770180215
885800000 1.6909713077143846
885900000 3.3179247267842293
886000000 1.8444398224416314
886100000 2.977140185006815
886200000 2.5587074686014644
886300000 2.5455170297765837
886400000 4.062879922056143
886500000 2.4498510289865143
886600000 1.6746056758067744
886700000 0.523238135144941
886800000 0.5255517749664591
886900000 0.5232371745146202
887000000 0.5168385026716709
887100000 0.5236213875331732
887200000 0.51846894268251
887300000 0.5189558256231884
887400000 0.5152257071560906
887500000 0.5201221695575278
887600000 0.5168212331398606
887700000 2.779162611092134
887800000 2.4050075118425824
887900000 4.1805920121009175
888000000 2.071343633546151
888100000 3.3998989604442205

best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 0.8490942039115761
909400000 0.4347958949987637
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.3229230880333006
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.3156306228243894
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.26917813002937835
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.8213180679067791
909900000 1.4136440167863429
910000000 1.2563377151711244
910100000 1.3672739748483875
910200000 1.2950575095631043
910300000 0.3802436845201618
91

921100000 1.1533653460852815
921200000 0.7386313450110615
921300000 1.273009691664862
921400000 0.8649945670961672
921500000 1.0904310162202178
921600000 1.1684164505128696
921700000 0.7576534492117034
921800000 1.353630358689624
921900000 0.9104304962946045
922000000 1.2131296529817528
922100000 0.40067324712556596
922200000 0.24429239271033804
922300000 0.21035068856448733
922400000 0.23359489359748564
922500000 0.24524653579487599
922600000 0.2401754394194305
922700000 0.22816903571452898
922800000 0.2432309416084157
922900000 0.23610853356650144
923000000 0.23611901983208655
923100000 0.21014186764444578
923200000 0.22534724409900242
923300000 0.6619061919289867
923400000 1.025467296472805
923500000 1.3036979032979368
923600000 1.2037900805220525
923700000 0.7798833115757612
923800000 1.1576374924466681
923900000 0.8830707607209145
924000000 1.0547005007437076
924100000 1.0704138493114803
924200000 0.7509901918784836
924300000 1.0334219034805325
924400000 0.7306640501631194
9245000

944800000 0.16439038939686615
944900000 0.1322057829395791
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.1125218486262805
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.059438972389393385
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.05842937357461453
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 1, 3] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 3, 1] 126
cases of this type: 1701
best so far: 0
type: [2, 7, 9, 1, 1] 126
cases of t

955400000 0.008756514209353923
955500000 0.006857481616954009
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.005244747738004393
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
cases of this type: 1323
best so far: 0
type: [6, 1, 21, 1, 1] 126
cases of this type: 441
best so far: 0
type: [6, 3, 1, 1, 7] 126
cases of this type: 7203
best so far: 0
type: [6, 3, 1, 7, 1] 126
cases of this type: 1029
best so far: 0
type: [6, 3, 7, 1, 1] 126
cases of this type: 147
best so far: 0
type: [6, 7, 1, 1, 3] 126
cases of this type: 567
best so far: 0
type: [6, 7, 1, 3, 1] 126
cases of this type: 189
best so far: 0
type: [6, 7, 3, 1, 1] 126
cases o